In [1]:
import os

In [2]:
if "COLAB_GPU" in os.environ:
 #   !pip install PyMuPDF # for reading PDFs using python
  #  !pip install tqdm # for progress bars
   # !pip install accelerate # for quantization model loading
    #!pip install bitsandbytes # for quantization models (less storage/memory usage)
    !pip install flash-attn --no-build-isolation # for faster attention computation

In [3]:
!pip install flash-attn --no-build-isolation

In [4]:
#!pip uninstall -y torch torchvision torchaudio transformers sentence-transformers
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!pip install -U transformers sentence-transformers

In [4]:
import os
import requests

pdf_path = "Human-Nutrition.pdf"

if not os.path.exists(pdf_path):
    print("Downloading PDF...")

    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    filename = pdf_path
    response = requests.get(url)

    if response.status_code == 200:
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
else:
    print(f"{pdf_path} already exists.")

Human-Nutrition.pdf already exists.


In [20]:
import fitz # PyMuPDF

from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    # minor formatting to clean up text
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> str:
    # Open the PDF file
    document = fitz.open(pdf_path)
    pages_and_texts = []

    # Iterate through each page and extract text
    for page_number, page in tqdm(enumerate(document)):
        #print(page_number, page)
        text = page.get_text()
        text = text_formatter(text)
        pages_and_texts.append({
        "page_number": page_number - 41,
        "page_char_count": len(text),
        "page_word_count": len(text.split(" ")),
        "Page_sentence_count_raw": len(text.split(". ")),
        "page_token_count": len(text)/4, # rough estimate: 1 token ~= 4 characters
        "text": text
        })

    return pages_and_texts


In [21]:
pages_and_texts = open_and_read_pdf(pdf_path = pdf_path)
pages_and_texts[:2]  # Display the first two pages' data

0it [00:00, ?it/s]

1208it [00:01, 672.68it/s]


[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'Page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'Page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [6]:
import random

random_pages = random.sample(pages_and_texts, k=3)
random_pages

[{'page_number': 275,
  'page_char_count': 1664,
  'page_word_count': 289,
  'Page_sentence_count_raw': 11,
  'page_token_count': 416.0,
  'text': 'long ingredient lists. On the front of food and beverages the  manufacturers may include claims such as “sugar-free,” “reduced  sugar,” “high fiber,” etc.. The Nutrition and Labeling Act of 1990  has defined for the food industry and consumers what these labels  mean (Table 4.4 “Food Labels Pertaining to Carbohydrates”).  Table 4.4 Food Labels Pertaining to Carbohydrates  Label  Meaning  Sugar-free  Contains less than 0.5 grams of sugar per serving  Reduced  sugar  Contains 25 percent less sugar than similar product  Less sugar  Contains 25 percent less sugar than similar product, and  was not altered by processing to become so  No sugars  added  No sugars added during processing  High fiber  Contains at least 20 percent of daily value of fiber in each  serving  A good  source of  fiber  Contains between 10 and 19 percent of the daily value

In [8]:
%pip install pandas

import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   ------------------ --------------------- 5.0/11.0 MB 19.7 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 24.7 MB/s  0:00:00

   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -

,page_number,page_char_count,page_word_count,Page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [10]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,Page_sentence_count_raw,page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,198.30,9.97,287.00
std,348.86,560.38,95.76,6.19,140.10
min,-41.00,0.00,1.00,1.00,0.00
25%,260.75,762.00,134.00,4.00,190.50
50%,562.50,1231.50,214.50,10.00,307.88
75%,864.25,1603.50,271.00,14.00,400.88
max,1166.00,2308.00,429.00,32.00,577.00


Potentially each page can be embedded with embedding models of input capacity ~ 300

### Chunking

One sentence can be one chunk or one page can also be one chunk

Top 1 or top 3 chunks which are closest to the prompt are passed to the LLM.